<a href="http://laf-fabric.readthedocs.org/en/latest/" target="_blank"><img align="left" src="images/laf-fabric-xsmall.png"/></a>
<a href="http://www.godgeleerdheid.vu.nl/etcbc" target="_blank"><img align="left" src="images/VU-ETCBC-xsmall.png"/></a>
<a href="http://www.persistent-identifier.nl/?identifier=urn%3Anbn%3Anl%3Aui%3A13-048i-71" target="_blank"><img align="left"src="images/etcbc4easy-small.png"/></a>
<a href="http://tla.mpi.nl" target="_blank"><img align="right" src="images/TLA-xsmall.png"/></a>
<a href="http://www.dans.knaw.nl" target="_blank"><img align="right"src="images/DANS-xsmall.png"/></a>

# Paragraphs from the px files

In [1]:
import re
from IPython.display import display, HTML, FileLinks
%load_ext autoreload
%autoreload 2
import laf
from laf.fabric import LafFabric
from etcbc.preprocess import prepare
from etcbc.extra import ExtraData
fabric = LafFabric()

  0.00s This is LAF-Fabric 4.4.7
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: http://shebanq-doc.readthedocs.org/en/latest/texts/welcome.html



## Create annotations from px file

In [2]:
version = '4b'
etcbc_source = 'etcbc{}'.format(version)
API=fabric.load(etcbc_source, '--', 'can', {
    "xmlids": {"node": True, "edge": False},
    "features": ('''
        otype monads number label
    ''',
    '''
    '''),
    "prepare": prepare,
}, verbose='DETAIL')
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.00s DETAIL: COMPILING m: UP TO DATE
  0.00s INFO: USING DATA COMPILED AT: 2015-05-04T13-46-20
  0.00s DETAIL: COMPILING a: UP TO DATE
  0.01s DETAIL: load main: G.node_anchor_min
  0.09s DETAIL: load main: G.node_anchor_max
  0.15s DETAIL: load main: G.node_sort
  0.20s DETAIL: load main: G.node_sort_inv
  0.65s DETAIL: load main: G.edges_from
  0.71s DETAIL: load main: G.edges_to
  0.79s DETAIL: load main: X. [node]  -> 
  2.37s DETAIL: load main: X. [node]  <- 
  3.11s DETAIL: load main: F.etcbc4_db_monads [node] 
  3.86s DETAIL: load main: F.etcbc4_db_otype [node] 
  4.53s DETAIL: load main: F.etcbc4_ft_number [node] 
  5.01s DETAIL: load main: F.etcbc4_sft_label [node] 
  5.04s LOGFILE=/Users/dirk/SURFdrive/laf-fabric-output/etcbc4b/can/__log__can.txt
  5.04s DETAIL: prep prep: G.node_sort
  5.11s DETAIL: prep prep: G.node_sort_inv
  5.62s INFO: DATA LOADED FROM SOURCE etcbc4b AND ANNOX -- FOR TASK can AT 2015-05-04T14-10-58


# Method to read px data

In [3]:
def read_px(px_file):
    msg("Making mappings between clause atoms in PX and nodes in LAF")
    ca_labn2id = {}
    ca_id2labn = {}
    for n in NN():
        otype = F.otype.v(n)
        if otype == 'verse':
            cur_label = F.label.v(n)
        elif otype == 'chapter':
            cur_subtract += cur_chapter_cas
            cur_chapter_cas = 0
        elif otype == 'book':
            cur_subtract = 0
            cur_chapter_cas = 0
        elif otype == 'clause_atom':
            cur_chapter_cas += 1
            nm = int(F.number.v(n)) - cur_subtract
            ca_labn2id[(cur_label, nm)] = n
            ca_id2labn[n] = (cur_label, nm)
    msg("End making mappings: {}={} clauses".format(len(ca_labn2id), len(ca_id2labn)))

    data = []
    not_found = set()
    px_handle = open(px_file)
    ln = 0
    can = 0
    featurescan = re.compile(r'0 0 (..) [0-9]+ LineNr\s*([0-9]+).*?Pargr:\s*([0-9.]+)')
    cur_label = None
    data = []
    for line in px_handle:
        ln += 1
        if line.strip()[0] != '*':
            cur_label = line[0:10]
            continue
        can += 1
        features = featurescan.findall(line)
        if len(features) == 0:
            msg("Warning: line {}: no instruction, LineNr, Pargr found".format(ln))
        elif len(features) > 1:
            msg("Warning: line {}: multiple instruction, LineNr, Pargr found".format(ln))
        else:
            feature = features[0]
            the_ins = feature[0]
            the_n = feature[1]
            the_para = feature[2]
            labn = (cur_label, int(the_n))
            if labn not in ca_labn2id:
                not_found.add(labn)
                continue
            data.append((ca_labn2id[labn], the_ins, the_n, the_para))
    px_handle.close()
    msg("Read {} paragraph annotations".format(len(data)))
    if not_found:
        msg("Could not find {} label/line entries in index: {}".format(len(not_found), sorted({lab for lab in not_found})))
    else:
        msg("All label/line entries found in index")
    return data


# Integrating the px data



In [4]:
px = ExtraData(API)
px.deliver_annots('px/px_data.{}'.format(etcbc_source), 'px', 'para', read_px, (
        ('etcbc4', 'px', 'instruction'),
        ('etcbc4', 'px', 'number_in_ch'),
        ('etcbc4', 'px', 'pargr'),
    ),
    {'title': 'Paragraph numbers', 'date': '2015'},
)

    20s Making mappings between clause atoms in PX and nodes in LAF
    22s End making mappings: 90315=90315 clauses
    23s Read 90274 paragraph annotations
    23s Could not find 43 label/line entries in index: [(' IKON07,30', 115), (' IKON07,31', 121), (' IKON07,32', 124), (' IKON07,33', 127), (' IKON07,34', 129), (' IKON07,35', 133), (' IKON07,36', 135), (' IKON07,37', 137), (' IKON07,38', 141), (' IKON07,39', 145), (' IKON07,40', 149), (' IKON07,41', 154), (' IKON07,42', 158), (' IKON07,43', 159), (' IKON07,44', 160), (' IKON07,45', 164), (' IKON07,46', 165), (' IKON07,47', 167), (' IKON07,48', 172), (' IKON07,49', 177), (' IKON07,50', 179), (' IKON07,51', 183), (' JER 32,24', 103), (' JER 32,25', 108), (' JER 32,26', 110), (' JER 32,27', 112), (' JER 32,28', 115), (' JER 32,29', 123), (' JER 32,30', 128), (' JER 32,31', 132), (' JER 32,32', 136), (' JER 32,33', 143), (' JER 32,34', 146), (' JER 32,35', 153), (' JER 32,36', 157), (' JER 32,37', 161), (' JER 32,38', 163), (' JER 32

## Checking: loading the new features

In [5]:
API=fabric.load(etcbc_source, 'px', 'can', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        otype number label
        instruction number_in_ch pargr
    ''',
    '''
    '''),
    "prepare": prepare,
}, verbose='DETAIL')
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.00s DETAIL: COMPILING m: UP TO DATE
  0.00s INFO: USING DATA COMPILED AT: 2015-05-04T13-46-20
  0.00s BEGIN COMPILE a: px
  0.00s DETAIL: load main: X. [node]  -> 
  1.28s DETAIL: load main: X. [e]  -> 
  3.52s DETAIL: load main: G.node_anchor_min
  3.58s DETAIL: load main: G.node_anchor_max
  3.63s DETAIL: load main: G.node_sort
  3.68s DETAIL: load main: G.node_sort_inv
  4.12s DETAIL: load main: G.edges_from
  4.19s DETAIL: load main: G.edges_to
  4.27s LOGFILE=/Users/dirk/SURFdrive/laf-fabric-data/etcbc4b/bin/A/px/__log__compile__.txt
  4.27s PARSING ANNOTATION FILES
  5.03s INFO: parsing para.xml
  9.55s INFO: END PARSING
         0 good   regions  and     0 faulty ones
         0 linked nodes    and     0 unlinked ones
         0 good   edges    and     0 faulty ones
     90274 good   annots   and     0 faulty ones
    270822 good   features and     0 faulty ones
     90274 distinct xml identifiers

  9.56s MODELING RESULT FILES
  9.56s I

## Inspecting all objects that got new features

In [6]:
ph = outfile('paras.txt')
cur_label = None
for n in NN():
    otype = F.otype.v(n)
    if otype == 'verse':
        cur_label = F.label.v(n)
    elif otype == 'clause_atom':
        nm = F.number_in_ch.v(n)
        if nm:
            ph.write("{}: instruction = {}; {}; para = {}\n".format(cur_label, nm, F.instruction.v(n), F.pargr.v(n)))
ph.close()

In [7]:
!head -n 100 {my_file('paras.txt')}

 GEN 01,01: instruction = 1; .N; para = 1
 GEN 01,02: instruction = 2; ..; para = 1
 GEN 01,02: instruction = 3; ..; para = 1
 GEN 01,02: instruction = 4; ..; para = 1
 GEN 01,03: instruction = 5; .#; para = 1.1
 GEN 01,03: instruction = 6; .q; para = 1.1.1
 GEN 01,03: instruction = 7; .#; para = 1.1.2
 GEN 01,04: instruction = 8; .#; para = 1.1.3
 GEN 01,04: instruction = 9; ..; para = 1.1.3
 GEN 01,04: instruction = 10; .#; para = 1.1.4
 GEN 01,05: instruction = 11; .#; para = 1.1.5
 GEN 01,05: instruction = 12; ..; para = 1.1.5
 GEN 01,05: instruction = 13; .#; para = 1.1.5.1
 GEN 01,05: instruction = 14; .#; para = 1.1.5.2
 GEN 01,05: instruction = 15; ..; para = 1.1.5.2
 GEN 01,06: instruction = 16; .#; para = 1.2
 GEN 01,06: instruction = 17; .q; para = 1.2.1
 GEN 01,06: instruction = 18; ..; para = 1.2.1
 GEN 01,07: instruction = 19; .#; para = 1.2.2
 GEN 01,07: instruction = 20; ..; para = 1.2.2
 GEN 01,07: instruction = 21; .e; para = 1.2.2
 GEN 01,07: instruction = 22; d.; pa